In [1]:
import json
import numpy as np
import gurobipy
from gurobipy import Model, GRB

In [2]:
with open('instances/toy_instance.json') as f:
    instance = json.load(f)

In [3]:
instance

{'horizon': 5,
 'qualifications': ['A', 'B', 'C'],
 'staff': [{'name': 'Olivia',
   'qualifications': ['A', 'B', 'C'],
   'vacations': []},
  {'name': 'Liam', 'qualifications': ['A', 'B'], 'vacations': [1]},
  {'name': 'Emma', 'qualifications': ['C'], 'vacations': [2]}],
 'jobs': [{'name': 'Job1',
   'gain': 20,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'B': 1, 'C': 1}},
  {'name': 'Job2',
   'gain': 15,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'B': 2}},
  {'name': 'Job3',
   'gain': 15,
   'due_date': 4,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'C': 2}},
  {'name': 'Job4',
   'gain': 20,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'B': 2, 'C': 1}},
  {'name': 'Job5',
   'gain': 10,
   'due_date': 5,
   'daily_penalty': 3,
   'working_days_per_qualification': {'C': 2}}]}

In [4]:
qualifications_set = set()
for employee in instance['staff']:
    qualifications_set |= set(employee['qualifications'])
qualifications = sorted(qualifications_set)

Nm = len(instance['staff'])
Nc = len(qualifications)
Np = len(instance['jobs'])
Nj = instance['horizon']

In [5]:
HasComp = np.zeros((Nm, Nc), dtype=np.int32)
Conge = np.zeros((Nm, Nj), dtype=np.int32)
NeedComp = np.zeros((Np, Nc), dtype=np.int32)
Rev = np.zeros(Np, dtype=np.int32)
Penalite = np.zeros(Np, dtype=np.int32)
Deadline = np.zeros(Np, dtype=np.int32)

for i, person in enumerate(instance['staff']):
    for qualif in person['qualifications']:
        k = qualifications.index(qualif)
        HasComp[i, k] = 1
    for vacation in person['vacations']:
        Conge[i, vacation-1] = 1
for j, job in enumerate(instance['jobs']):
    for qualif, days in job['working_days_per_qualification'].items():
        k = qualifications.index(qualif)
        NeedComp[j, k] = days
    Rev[j] = job['gain']
    Penalite[j] = job['daily_penalty']
    Deadline[j] = job['due_date']

In [6]:
m = Model("CompuOpti")

# Variables de decision
Travail = m.addMVar((Nm, Np, Nc, Nj), vtype=GRB.BINARY, name="Travail")
ATravail = m.addMVar((Np, Nj), vtype=GRB.BINARY, name="ATravail")
Realise = m.addMVar(Nj, vtype=GRB.BINARY, name='Realise')
Debut = m.addMVar(Np, lb=0, ub=Nj-1, vtype=GRB.INTEGER, name='Debut')
Fin = m.addMVar(Np, lb=0, ub=Nj-1, vtype=GRB.INTEGER, name='Fin')
DureeMax = m.addVar(lb=0, ub=Nj, name='DureeMax')
AffecteProj = m.addMVar((Nm, Np), vtype=GRB.BINARY, name='AffecteProj')

Retard = Fin - Deadline


# Contrainte de qualification
for j in range(Np):
    for l in range(Nj):
        m.addConstr(Travail[:, j, :, l] <= HasComp)

# Contrainte d’unicité de l’affectation
for i in range(Nm):
    for l in range(Nj):
        m.addConstr(Travail[i, :, :, l].sum() <= 1)

# Contrainte de congé
for j in range(Np):
    for k in range(Nc):
        m.addConstr(Travail[:, j, k, :] <= 1 - Conge)

# Contraintes d’unicité de la réalisation d’un projet et de couverture des qualifications
for j in range(Np):
    for k in range(Nc):
        m.addConstr(Travail[:, j, k, :].sum() == Realise[j] * NeedComp[j, k])

# Contraintes sur la variable ATravail
for i in range(Nm):
    for k in range(Nc):
        m.addConstr(ATravail >= Travail[i, :, k, :])

# Contraintes sur la variable AffectéPro
for k in range(Nc):
    for l in range(Nj):
        m.addConstr(AffecteProj >= Travail[:, :, k, l])

# Contraintes sur la durée d’un projet
for l in range(Nj):
    m.addConstr(Debut <= l * ATravail[:, l] + Nj * (1 - ATravail[:, l]))
    m.addConstr(Fin >= l * ATravail[:, l])
m.addConstr(DureeMax >= Fin - Debut + 1)

# Fonctions objectifs
f1 = -(Rev.T @ Realise - Penalite.T @ Retard)
f2 = AffecteProj.sum()
f3 = DureeMax * 1

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-11


In [7]:
m.params.outputflag = 0
m.setObjective(f1)
m.optimize()

In [8]:
f1.getValue()

array(-94.)

In [9]:
Travail.X

array([[[[-0., -0., -0., -0., -0.],
         [-0., -0., -0.,  1., -0.],
         [-0., -0., -0., -0., -0.]],

        [[ 1., -0., -0., -0., -0.],
         [-0.,  1., -0., -0., -0.],
         [ 0.,  0.,  0.,  0.,  0.]],

        [[-0., -0., -0., -0., -0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [-0., -0., -0., -0., -0.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [-0., -0.,  1., -0.,  0.],
         [-0., -0., -0., -0., -0.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [-0., -0., -0., -0., -0.]]],


       [[[ 0., -0., -0.,  1., -0.],
         [ 0., -0., -0., -0., -0.],
         [ 0.,  0.,  0.,  0.,  0.]],

        [[ 0., -0., -0., -0., -0.],
         [ 0.,  1., -0., -0., -0.],
         [ 0.,  0.,  0.,  0.,  0.]],

        [[ 0., -0., -0., -0.,  1.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0., -0.,  1., -0., -0.],
         [ 0.,  0.,  0.,  0.,  0.]],

        